# Travel Destination Recommendation Application

### Project Goal
1. To recommend travel localtions based on users' requirements
2. Prelimilary settings: focusing on flight tickets, budgetary, cost driven
3. Extended: hotel prices, weathers, special occasions, ongoing status...

### Users
Everyone who loves traveling

### Use Cases
1. Real time recommendation: When the user enter the current location (city), return top 10 recommendation from low to high flight ticket price
2. In a certain timeframe, recommend top 10 traveling location from current location
3. Given a destination, return the rank of months to travel from current location

In [1]:
# Build a web scraping bot for best ticket price based on Kayak

from time import sleep, strftime
from random import randint
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
# Use Chromedriver to open Chrome

chromedriver_path = '/usr/local/Caskroom/chromedriver/83.0.4103.39/chromedriver'
driver = webdriver.Chrome(executable_path=chromedriver_path) # This will open the Chrome window
sleep(2)

In [3]:
# Define page scraping function

def page_scrape():
    """This is the function that does the scraping!"""
    
    xp_sections = '(//*[@class="section duration allow-multi-modal-icons"])[position()<3]'
    sections = driver.find_elements_by_xpath(xp_sections)
    sections_list = [value.text for value in sections]
    section_a_list = sections_list[::2]  # This is to separate the two flights
    section_b_list = sections_list[1::2] # This is to separate the two flights
    
    # If running into reCaptcha, add a sleep
    # If returned an empty list, use SystemExit, to test everything from the start
#    if section_a_list == []:
#        raise SystemExit
        
    # letter A for outbound flight; letter B for inbound flight
    a_duration = []
    a_section_names = []
    for n in section_a_list:
        # Separate time from the cities
        a_section_names.append(''.join(n.split()[2:5]))
        a_duration.append(''.join(n.split()[0:2]))
    b_duration = []
    b_section_names = []
    for n in section_b_list:
        # Separate time from the cities
        b_section_names.append(''.join(n.split()[2:5]))
        b_duration.append(''.join(n.split()[0:2]))
        
    xp_dates = '(//div[@class="section date"])[position()<3]'
    dates = driver.find_elements_by_xpath(xp_dates)
    dates_list = [value.text for value in dates]
    a_date_list = dates_list[::2]
    b_date_list = dates_list[1::2]
    # Separate the weekday from the day
    a_day = [value.split()[0] for value in a_date_list]
    a_weekday = [value.split()[1] for value in a_date_list]
    b_day = [value.split()[0] for value in b_date_list]
    b_weekday = [value.split()[1] for value in b_date_list]
    
    # Get the prices
    xp_prices = '(//span[contains(@id,"price-text") and contains(@class,"price-text")])[position()=1]'
    prices = driver.find_elements_by_xpath(xp_prices)
    prices_list = [price.text.replace('$', '') for price in prices if price.text != '']
#   prices_list = list(map(int, prices_list))
    
    # The stops are a big list with one leg at the even index and second leg on odd index
    xp_stops = '(//div[@class="section stops"]/div[1])[position()<3]'
    stops = driver.find_elements_by_xpath(xp_stops)
    stops_list = [stop.text[0].replace('n', '0') for stop in stops]
    a_stop_list = stops_list[::2]
    b_stop_list = stops_list[1::2]
    
    xp_stops_cities = '(//div[@class="section stops"]/div[2])[position()<3]'
    stops_cities = driver.find_elements_by_xpath(xp_stops_cities)
    stops_cities_list = [stop.text for stop in stops_cities]
    a_stop_name_list = stops_cities_list[::2]
    b_stop_name_list = stops_cities_list[1::2]
    
    # This part gets the airline company and the departure and arrival times for both legs
    xp_schedule = '(//div[@class="section times"])[position()<3]'
    schedules = driver.find_elements_by_xpath(xp_schedule)
    hours_list = []
    carrier_list = []
    for schedule in schedules:
        hours_list.append(schedule.text.split('\n')[0])
        carrier_list.append(schedule.text.split('\n')[1])
    # Split the hours and carriers, between a and b legs
    a_hours = hours_list[::2]
    a_carrier = carrier_list[::2]
    b_hours = hours_list[1::2]
    b_carrier = carrier_list[1::2]
    
    # Create dataframe
#    cols = (['Out Day', 'Out Time', 'Out Weekday', 'Out Airline', 'Out Cities', 'Out Duration', 'Out Stops', 'Out Stop Cities',
#            'Return Day', 'Return Time', 'Return Weekday', 'Return Airline', 'Return Cities', 'Return Duration', 'Return Stops', 'Return Stop Cities',
#            'Price'])
    
    flights_df = pd.DataFrame({'Out Day': a_day,
                               'Out Weekday': a_weekday,
                               'Out Duration': a_duration,
                               'Out Cities': a_section_names,
                               'Return Day': b_day,
                               'Return Weekday': b_weekday,
                               'Return Duration': b_duration,
                               'Return Cities': b_section_names,
                               'Out Stops': a_stop_list,
                               'Out Stop Cities': a_stop_name_list,
                               'Return Stops': b_stop_list,
                               'Return Stop Cities': b_stop_name_list,
                               'Out Time': a_hours,
                               'Out Airline': a_carrier,
                               'Return Time':b_hours,
                               'Return Airline': b_carrier,
                               'Price': prices_list})
    
    data_only = flights_df.values
    
#   flights_df['timestamp'] = strftime("%Y%m%d-%H%M") # so we can know when it was scraped
    return data_only

In [4]:
# Call Kayak

def start_kayak(city_from, city_to, date_start, date_end):
    """City codes - it's IATA codes!
       Date format - YYYY-MM-DD"""
    
    kayak = 'http://www.kayak.com/flights/' + city_from + '-' + city_to + '/' + date_start + '-flexible/' + date_end + '-flexible?sort=bestflight_a'
    driver.get(kayak)
    sleep(randint(8,10))
    
    # Sometimes a popup shows up, so we can use a try statement to check it and close (COVID19)
    try:
        xp_popup_close = '//button[contains(@id,"dialog-close") and contains(@class,"Button-No-Standard-Style close")]'
        driver.find_elements_by_xpath(xp_popup_close)[8].click()
    except Exception as e:
        pass
    sleep(randint(8,10))
    
    # Sometimes a popup shows up, so we can use a try statement to check it and close
    try:
        xp_popup_close = '//button[contains(@id,"dialog-close") and contains(@class,"Button-No-Standard-Style close")]'
        driver.find_elements_by_xpath(xp_popup_close)[9].click()
    except Exception as e:
        pass
    sleep(randint(8,10))
    
    
    # Scape the "best" price
    df_flights_best = page_scrape()
#   df_flights_best['sort'] = 'best'
    sleep(randint(3,6))

    # Let's also get the lowest prices from the matrix on top
#    matrix = driver.find_elements_by_xpath('//*[contains(@id,"FlexMatrixCell")]')
#    matrix_prices = [price.text.replace('$','') for price in matrix]
#    matrix_prices = list(map(int, matrix_prices))
#    matrix_min = min(matrix_prices)
#    matrix_avg = sum(matrix_prices)/len(matrix_prices)
    
    print(df_flights_best)

In [5]:
# Iterating through airports

#def iteration(my_airport, depart_date, return_date):
#    """This is where users enter their local airport!"""
my_airport = 'SFO'
depart_date = '2020-07-20'
return_date = '2020-07-22'

airports = pd.read_csv('airport_data.csv')
airports_code = airports['IATA']

cols = ['Out Day', 'Out Time', 'Out Weekday', 'Out Airline', 'Out Cities', 'Out Duration', 'Out Stops', 'Out Stop Cities',
            'Return Day', 'Return Time', 'Return Weekday', 'Return Airline', 'Return Cities', 'Return Duration', 'Return Stops', 'Return Stop Cities',
            'Price']
df = {'data':[]}

for code in airports_code:
    
    df['data'].append(start_kayak(my_airport, code, depart_date, return_date))
    
df
#result_df = pd.DataFrame.from_dict(df['data'], orient=index, columns=cols)
#result_df

[['7/21' 'Tue' '21h18m' 'SFO‐BHM' '7/23' 'Thu' '12h23m' 'BHM‐SFO' '2'
  'LAX, DFW' '2' 'DFW, LAX' '1:30 pm – 12:48 pm +1' 'American Airlines'
  '11:35 am – 9:58 pm' 'American Airlines' '570']]
[['7/22' 'Wed' '12h20m' 'SFO‐ANC' '7/24' 'Fri' '14h02m' 'ANC‐SFO' '2'
  'LAX, SEA' '2' 'SEA, LAX' '7:00 am – 6:20 pm' 'Delta'
  '6:15 am – 9:17 pm' 'Delta' '468']]
[['7/23' 'Thu' '5h21m' 'SFO‐PHX' '7/24' 'Fri' '11h15m' 'PHX‐SFO' '1'
  'LAS' '1' 'DEN' '8:55 am – 2:16 pm' 'Frontier' '6:05 am – 5:20 pm'
  'Frontier' '128']]
[['7/19' 'Sun' '9h36m' 'SFO‐TUS' '7/24' 'Fri' '15h26m' 'TUS‐SFO' '1'
  'LAS' '1' 'LAS' '8:55 am – 6:31 pm' 'Frontier, Allegiant Air'
  '4:39 pm – 8:05 am +1' 'Allegiant Air, Frontier' '214']]
[['7/21' 'Tue' '32h38m' 'SFO‐LIT' '7/23' 'Thu' '6h06m' 'LIT‐SFO' '2'
  'DEN, LAX' '1' 'DEN' '12:59 am – 11:37 am +1' 'Multiple Airlines'
  '7:42 pm – 11:48 pm' 'Frontier' '320']]
[['7/23' 'Thu' '5h19m' 'SFO‐BUR' '7/25' 'Sat' '14h54m' 'BUR‐SFO' '1'
  'LAS' '1' 'LAS' '10:52 am – 4:11 pm' 'Fron

[['7/22' 'Wed' '1h08m' 'SFO‐RNO' '7/22' 'Wed' '1h07m' 'RNO‐SFO' '0' ''
  '0' '' '11:15 am – 12:23 pm' 'United Airlines' '3:58 pm – 5:05 pm'
  'United Airlines' '340']]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


{'data': [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None]}

In [5]:
# Testing

# city_from = input('From which city? ')
# city_to = input('Where to? ')
# date_start = input('Search around which departure date? Please use YYYY-MM-DD format only ')
# date_end = input('Return when? Please use YYYY-MM-DD format only ')

city_from = 'SFO'
city_to = 'LAX'
date_start = '2020-07-10'
date_end = '2020-07-12'

start_kayak(city_from, city_to, date_start, date_end)


loading more...
starting the first scrape...


SystemExit: 

/Users/piapiahong/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [23]:
# debugging area

chromedriver_path = '/usr/local/Caskroom/chromedriver/83.0.4103.39/chromedriver'
driver = webdriver.Chrome(executable_path=chromedriver_path) # This will open the Chrome window
sleep(2)

kayak = 'https://www.kayak.com/flights/SFO-LAX/2020-07-20-flexible/2020-07-22-flexible?sort=bestflight_a'
driver.get(kayak)
sleep(randint(8,10))

try:
    xp_popup_close = '//button[contains(@id,"dialog-close") and contains(@class,"Button-No-Standard-Style close")]'
    driver.find_elements_by_xpath(xp_popup_close)[8].click()
except Exception as e:
    pass
sleep(randint(8,10))
print('loading more...')
    
xp_prices = '(//span[contains(@id,"price-text") and contains(@class,"price-text")])[position()=1]'
prices = driver.find_elements_by_xpath(xp_prices)
prices_list = [price.text.replace('$','') for price in prices if price.text != '']
#prices_list = list(map(int, prices_list))

print(prices_list)

loading more...
['120']


In [19]:
airports = pd.read_csv('airport_data.csv')
print(airports['IATA'])

0      BHM
1      ANC
2      PHX
3      TUS
4      LIT
      ... 
98     PPG
99     GUM
100    SPN
101    SJU
102    STT
Name: IATA, Length: 103, dtype: object


In [1]:
a=[]
for i in range(5):    
    a.append(i)

a

[0, 1, 2, 3, 4]